In [ ]:
# Installation
!pip install --upgrade --quiet google-genai

In [ ]:
!pip install gTTs --quiet

In [ ]:
!pip install playsound --quiet

In [ ]:
# API Key setup
from google.colab import userdata
GEMINI_API_KEY: str = userdata.get('GOOGLE_API_KEY')
if(GEMINI_API_KEY):
  print("API Key found")
else:
  print("API Key not found")

API Key found


In [ ]:
from google.genai.types import GenerateContentResponse
from IPython.display import display, Markdown

In [ ]:
# Initilize and configure the client
# Select the model
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

In [ ]:
video_file_name = "HussainDurrani_WhatsApp Video.mp4"

In [ ]:
# Upload all the videos using the File API.
# You can find more details about how to use it in the Get Started notebook.
# This can take a couple of minutes as the videos will need to be processed and tokenized.

import time


def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video(video_file_name)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/wi2ty1obcuxw


In [ ]:
from google.genai.types import Content, Part
prompt = """For overall scene in this video,
            generate captions that describe the scene along with any spoken text. Explain the scenery in detail & tell about the persons expressions as well
            Write only captions and speech. No extra elements. Do not write the timecode & brackets. Don't time code during the speech.
         """

video = my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)

Markdown(response.text)

A young man in a black outfit stands in front of a tiered golden chandelier. The man has a solemn expression while he is speaking.

Hello, this is Hussain Durrani. I'm currently pursuing BSCS from UCP and also doing a course of gentic AI engineering from PIAIC. Thank you for listening and have a nice day. Bye.

In [ ]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio


tts = gTTS(text=response.text, lang='en')

with open('output.mp3', 'wb') as f:
  for chunk in tts.stream():
    f.write(chunk)

display(Audio('output.mp3', autoplay=True))